In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
import csv
import os
from datetime import datetime
import numpy as np

#### Steps:

1. Calculate average cardio score of each divison so you have a standard
2. Calculate cardio score for a fighter given as input
3. Compare fighters cardio score to the average for that division, and assign a final standardized cardio score

In [2]:
df_fight_results = pd.read_csv('./data/github/ufc_fight_results.csv')
df_fight_stats = pd.read_csv('./data/github/ufc_fight_stats.csv')

divisions = [
    "Flyweight Bout",
    "Bantamweight Bout",
    "Featherweight Bout",
    "Lightweight Bout",
    "Welterweight Bout",
    "Middleweight Bout",
    "Light Heavyweight Bout",
    "Heavyweight Bout",
    "Women's Strawweight Bout",
    "Women's Flyweight Bout",
    "Women's Bantamweight Bout",
    "Women's Featherweight Bout"
]


In [3]:
# Calculate Total Fight Time In Minutes

df_fight_results['TIME_MINUTES'] = df_fight_results['TIME'].apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1]) / 60)
df_fight_results['TOTAL_FIGHT_TIME'] = (df_fight_results['ROUND'] - 1) * 5 + df_fight_results['TIME_MINUTES']
df_fight_results[['ROUND', 'TIME', 'TOTAL_FIGHT_TIME']].head()
df_fight_results.to_csv('./data/github/ufc_fight_results.csv', index=False)


In [4]:
# Calculate cardio score for all fighters in a division
# Based off Simple Aggregation: It aggregates the changes in total strikes attempted and takedown 
# attempts for each fighter across their bouts to compute a cardio score. This is a straightforward 
# sum and average calculation without applying any complex modeling or statistical techniques.

division = 'Welterweight'

lightweight_bouts_df = df_fight_results[df_fight_results['WEIGHTCLASS'].str.contains(division, case=False, na=False)]
lightweight_fighter_names = set() # Parsing the 'BOUT' column to extract fighter names correctly

for bout in lightweight_bouts_df['BOUT']:
    fighters = bout.split(' vs. ')  # Splitting the bout string to extract fighter names
    fighter_1, fighter_2 = fighters[0].strip(), fighters[1].strip()  # Removing any leading/trailing whitespace
    lightweight_fighter_names.add(fighter_1)
    lightweight_fighter_names.add(fighter_2)

lightweight_fighter_names = list(lightweight_fighter_names) # Convert the set to a list to facilitate processing
len(lightweight_fighter_names), lightweight_fighter_names[:5] # Display the number of unique lightweight fighters found and the first few names for verification

lightweight_fighter_cardio_scores = {}

for fighter_name in lightweight_fighter_names:
    fighter_bouts = df_fight_stats[df_fight_stats['FIGHTER'] == fighter_name]
    unique_bouts = fighter_bouts['BOUT'].unique()
    fighter_cardio_scores = []
    for bout in unique_bouts:
        bout_df = df_fight_stats[(df_fight_stats['FIGHTER'] == fighter_name) & (df_fight_stats['BOUT'] == bout)].copy()
        bout_df.loc[:, 'TOTAL STR. Attempted'] = bout_df['TOTAL STR.'].str.split(' of ').str[1].astype(int)
        bout_df.loc[:, 'TD Attempted'] = bout_df['TD'].str.split(' of ').str[1].astype(int)
        str_changes = bout_df['TOTAL STR. Attempted'].diff().fillna(0).tolist()
        td_changes = bout_df['TD Attempted'].diff().fillna(0).tolist()
        bout_score = sum(str_changes[1:]) + sum(td_changes[1:])
        fighter_cardio_scores.append(bout_score)
    overall_score = sum(fighter_cardio_scores) / len(fighter_cardio_scores) if fighter_cardio_scores else 0
    lightweight_fighter_cardio_scores[fighter_name] = overall_score
    
# Print Average of Division
average_lightweight_cardio_score = sum(lightweight_fighter_cardio_scores.values()) / len(lightweight_fighter_cardio_scores)
print('Average of Division:', average_lightweight_cardio_score)

# Print from Best to Worst
cardio_scores_df = pd.DataFrame(list(lightweight_fighter_cardio_scores.items()), columns=['Fighter', 'Cardio Score'])
cardio_scores_df = cardio_scores_df.sort_values(by='Cardio Score', ascending=False)
print('\nAll from Division: \n\n', cardio_scores_df.to_string(index=False))


Average of Division: -1.44911658944502

All from Division: 

                   Fighter  Cardio Score
           Danny Mitchell     64.500000
            Carlton Minus     51.500000
        Aleksei Kunchenko     43.500000
        Sergey Khandozhko     37.666667
         Tristan Connelly     36.000000
           Rolando Bedoya     36.000000
                 Sai Wang     35.500000
            Jonny Parsons     33.000000
             Jim Wallhead     31.500000
                Emil Meek     30.250000
           Gasan Umalatov     25.250000
             Jessin Ayari     23.750000
            Nicolas Dalby     23.461538
                Jon Fitch     22.058824
              Chris Spang     21.000000
            Frank Camacho     21.000000
           Tom Gallicchio     21.000000
                Ian Garry     20.333333
         Steve Montgomery     20.000000
         Stephen Thompson     19.736842
        Yoshihiro Akiyama     19.285714
           Matthew Riddle     19.250000
              Jeff

In [5]:
# Print by ABC Order

cardio_scores_df = pd.DataFrame(list(lightweight_fighter_cardio_scores.items()), columns=['Fighter', 'Cardio Score'])
cardio_scores_df = cardio_scores_df.sort_values(by='Fighter', ascending=True)
print(cardio_scores_df.to_string(index=False))


                  Fighter  Cardio Score
              AJ Fletcher    -18.000000
              Aaron Riley     11.444444
            Aaron Simpson    -16.909091
     Abdul Razak Alhassan     -3.000000
    Abubakar Nurmagomedov      7.000000
              Adam Fugitt     -8.750000
             Adlan Amagov     16.500000
           Adrian Serrano    -13.000000
             Akihiro Gono      3.333333
              Alan Jouban      8.076923
             Albert Cheng      0.000000
           Albert Tumenov      1.875000
             Alberto Mina     -0.750000
        Aleksei Kunchenko     43.500000
              Alex Garcia     -3.600000
           Alex Karalexis    -10.666667
              Alex Morono     -1.904762
            Alex Oliveira     -5.761905
               Alex Reyes      0.000000
       Alexander Yakovlev      7.444444
         Alexandre Barros    -19.000000
   Alvaro Herrera Mendoza     -1.250000
            Amilcar Alves      4.000000
            Amir Sadollah      5.454545
